In [1]:
import os
import pandas as pd

# データの保存先ファイル
output_file = 'data/tcfd_target.csv'

# 読み取るデータ項目
columns_to_extract = ["書類管理番号", "書類名", "EDINET(ファンド)コード", "提出者(ファンド)", "提出者法人番号"]
# 追加のデータ項目
columns_to_extract.extend(["XBRLファイル名", "XBRLファイルパス"])

# 結果を格納するリスト
result_data = []

# data/XBRLフォルダのパス
base_dir = 'data/XBRL'

# Xbrl_Searchで始まるフォルダを取得
folders = [folder for folder in os.listdir(base_dir) if folder.startswith('Xbrl_Search')]

for folder in folders:
    folder_path = os.path.join(base_dir, folder)
    csv_file = os.path.join(folder_path, 'XbrlSearchDlInfo.csv')
    
    if os.path.exists(csv_file):
        # CSVファイルを読み込む (Shift_JISエンコーディング)
        df = pd.read_csv(csv_file, header=1, encoding='shift_jis')
        if not df.empty:
            # 指定された列のデータを取得
            data_row = df.iloc[0][columns_to_extract[:-2]].to_dict()
            # 書類管理番号を取得
            doc_number = data_row["書類管理番号"]
            
            # XBRLファイルのパス
            xbrl_folder = os.path.join(folder_path, doc_number, 'XBRL', 'PublicDoc')
            if os.path.exists(xbrl_folder):
                # XBRLファイル名を取得
                xbrl_files = [file for file in os.listdir(xbrl_folder) if file.endswith('.xbrl')]
                if xbrl_files:
                    # 最初のXBRLファイル名とそのパスを取得
                    xbrl_file_name = xbrl_files[0]
                    data_row["XBRLファイル名"] = xbrl_file_name
                    data_row["XBRLファイルパス"] = os.path.join(xbrl_folder, xbrl_file_name)
                else:
                    data_row["XBRLファイル名"] = ""
                    data_row["XBRLファイルパス"] = ""
            else:
                data_row["XBRLファイル名"] = ""
                data_row["XBRLファイルパス"] = ""
            
            # 結果に追加
            result_data.append(data_row)

# 結果をデータフレームに変換
result_df = pd.DataFrame(result_data, columns=columns_to_extract)

# 結果をCSVファイルに保存
result_df.to_csv(output_file, index=False, encoding='utf-8-sig')

#print(f"{len(result_data)} 件のデータを {output_file} に保存しました。")
result_df.head(30)

,書類管理番号,書類名,EDINET(ファンド)コード,提出者(ファンド),提出者法人番号,XBRLファイル名,XBRLファイルパス
0,S100R5K4,有価証券報告書,E00691,特種東海製紙株式会社,3080001014336,jpcrp030000-asr-001_E00691-000_2023-03-31_01_2...,data/XBRL/Xbrl_Search_20240419_152847/S100R5K4...
1,S100R87F,有価証券報告書,E00642,王子ホールディングス株式会社,5010001034743,jpcrp030000-asr-001_E00642-000_2023-03-31_01_2...,data/XBRL/Xbrl_Search_20240419_152909/S100R87F...
2,S100R8UQ,有価証券報告書,E11873,日本製紙株式会社,8011501009422,jpcrp030000-asr-001_E11873-000_2023-03-31_01_2...,data/XBRL/Xbrl_Search_20240419_152923/S100R8UQ...
3,S100R82J,有価証券報告書,E00644,三菱製紙株式会社,8010001008769,jpcrp030000-asr-001_E00644-000_2023-03-31_01_2...,data/XBRL/Xbrl_Search_20240419_152939/S100R82J...
4,S100R6PJ,有価証券報告書,E00645,北越コーポレーション株式会社,6110001023149,jpcrp030000-asr-001_E00645-000_2023-03-31_01_2...,data/XBRL/Xbrl_Search_20240419_152951/S100R6PJ...
5,S100R7Q8,有価証券報告書,E00649,中越パルプ工業株式会社,3010001034877,jpcrp030000-asr-001_E00649-000_2023-03-31_01_2...,data/XBRL/Xbrl_Search_20240419_153004/S100R7Q8...
6,S100R8GL,有価証券報告書,E00660,大王製紙株式会社,9500001014345,jpcrp030000-asr-001_E00660-000_2023-03-31_01_2...,data/XBRL/Xbrl_Search_20240419_153021/S100R8GL...
7,S100R69D,有価証券報告書,E00659,レンゴー株式会社,1120001036880,jpcrp030000-asr-001_E00659-000_2023-03-31_01_2...,data/XBRL/Xbrl_Search_20240419_153035/S100R69D...
8,S100R35K,有価証券報告書,E00669,株式会社トーモク,7010001024692,jpcrp030000-asr-001_E00669-000_2023-03-31_01_2...,data/XBRL/Xbrl_Search_20240419_153051/S100R35K...
9,S100T5K9,有価証券報告書,E00674,ザ・パック株式会社,5120001012382,jpcrp030000-asr-001_E00674-000_2023-12-31_01_2...,data/XBRL/Xbrl_Search_20240419_164914/S100T5K9...


In [2]:
target_doc_number="S100TOGW"
fundcode = result_df.loc[result_df["書類管理番号"] == target_doc_number, "EDINET(ファンド)コード"].values[0]
file_path = result_df.loc[result_df["書類管理番号"] == target_doc_number, "XBRLファイルパス"].values[0]


In [3]:
from bs4 import BeautifulSoup
import html

# XBRLファイルを読み込む
#file_path = 'data/XBRL/jpcrp030000-asr-001_E01739-000_2023-03-31_01_2023-06-29.xbrl'
#file_path = 'data/XBRL/Xbrl_Search_20240419_152909/S100R87F/XBRL/PublicDoc/jpcrp030000-asr-001_E00642-000_2023-03-31_01_2023-06-29.xbrl'

with open(file_path, 'r', encoding='utf-8') as file:
    xbrl_content = file.read()

# エスケープ文字を元のタグに戻す
decoded_content = html.unescape(xbrl_content)

# BeautifulSoupでパース
soup = BeautifulSoup(decoded_content, 'xml')

# 各タグの属性情報を削除する
def remove_tag_attributes(tag):
    for attribute in list(tag.attrs):
        del tag[attribute]

# <tr>タグ、<td>タグで囲まれたテキストから改行コードを除去する
def remove_newlines_within_tags(tag):
    if tag.name == 'tr':
        for td_tag in tag.find_all('td'):
            for child in td_tag.children:
                if child.name == 'pstyle':  # <pstyle>タグは<p>に変換してスタイル属性を削除
                    child.name = 'p'
                    del child['style']
            # テキストノードの改行を除去
            td_tag.string = td_tag.get_text(separator="", strip=True)

# すべてのタグを削除し、コンテンツのみを残す
def remove_all_tags(tag):
    #print("Before removing tags:")
    #print(tag.prettify())

    # コンテンツごとに改行を挿入してタグを削除
    content_text = ''.join(str(child) for child in tag.contents if isinstance(child, BeautifulSoup))

    # タグを削除する
    tag.extract()

    #print("\nAfter removing tags:")
    print(content_text)

# 対象タグを定義
element_tag='jpcrp030000-asr_'+fundcode+'-000:DisclosureOfSustainabilityRelatedFinancialInformationTextBlock'

print(element_tag)
# 特定の要素を見つける
elements = soup.find_all(element_tag)

# テーブル内容の各要素に対して処理を適用
for i, elem in enumerate(elements):
    # 各タグの属性情報を削除する
    for tag in elem.find_all():
        remove_tag_attributes(tag)
    # <tr>タグ、<td>タグで囲まれたテキストから改行コードを除去する
    for tag in elem.find_all(['tr', 'td']):
        remove_newlines_within_tags(tag)

    # すべてのタグを削除し、コンテンツのみを残す
    remove_all_tags(elem)

    # 処理後のテキストを出力する
    processed_text = elem.get_text(separator="\n", strip=True)
    if processed_text.strip():  # テキストが空でない場合のみ出力する
        #print(f"\nProcessed Text for Element {i}:")
        print(processed_text)
        #print("----------------------------------")

# 必要に応じて、修正されたXMLをファイルに保存する場合
# with open('processed_xbrl.xml', 'w', encoding='utf-8') as output_file:
#     output_file.write(soup.prettify())



jpcrp030000-asr_E01739-000:DisclosureOfSustainabilityRelatedFinancialInformationTextBlock


In [4]:
from bs4 import BeautifulSoup
import html

def extract_sustainable_info(fundcode, file_path):

	with open(file_path, 'r', encoding='utf-8') as file:
	    xbrl_content = file.read()

	# エスケープ文字を元のタグに戻す
	decoded_content = html.unescape(xbrl_content)

	# BeautifulSoupでパース
	soup = BeautifulSoup(decoded_content, 'xml')

	# 各タグの属性情報を削除する
	def remove_tag_attributes(tag):
	    for attribute in list(tag.attrs):
	        del tag[attribute]

	# <tr>タグ、<td>タグで囲まれたテキストから改行コードを除去する
	def remove_newlines_within_tags(tag):
	    if tag.name == 'tr':
	        for td_tag in tag.find_all('td'):
	            for child in td_tag.children:
	                if child.name == 'pstyle':  # <pstyle>タグは<p>に変換してスタイル属性を削除
	                    child.name = 'p'
	                    del child['style']
	            # テキストノードの改行を除去
	            td_tag.string = td_tag.get_text(separator="", strip=True)

	# すべてのタグを削除し、コンテンツのみを残す
	def remove_all_tags(tag):
	    #print("Before removing tags:")
	    #print(tag.prettify())

	    # コンテンツごとに改行を挿入してタグを削除
	    content_text = ''.join(str(child) for child in tag.contents if isinstance(child, BeautifulSoup))

	    # タグを削除する
	    tag.extract()

	    #print("\nAfter removing tags:")
	    #print(content_text)

	# 対象タグを定義
	element_tag='jpcrp030000-asr_'+fundcode+'-000:DisclosureOfSustainabilityRelatedFinancialInformationTextBlock'

	
    #print(element_tag)
	# 特定の要素を見つける
	elements = soup.find_all(element_tag)

	# テーブル内容の各要素に対して処理を適用
	for i, elem in enumerate(elements):
	    # 各タグの属性情報を削除する
	    for tag in elem.find_all():
	        remove_tag_attributes(tag)
	    # <tr>タグ、<td>タグで囲まれたテキストから改行コードを除去する
	    for tag in elem.find_all(['tr', 'td']):
	        remove_newlines_within_tags(tag)

	    # すべてのタグを削除し、コンテンツのみを残す
	    remove_all_tags(elem)

	    # 処理後のテキストを出力する
	    processed_text = elem.get_text(separator="\n", strip=True)


In [5]:
print(fundcode)
print(file_path)
print(element_tag)
print(processed_text)

E01739
data/XBRL/Xbrl_Search_20240709_152741/S100TOGW/XBRL/PublicDoc/jpcrp030000-asr-001_E01739-000_2024-03-31_01_2024-06-25.xbrl
jpcrp030000-asr_E01739-000:DisclosureOfSustainabilityRelatedFinancialInformationTextBlock


NameError: name 'processed_text' is not defined

In [8]:
from bs4 import BeautifulSoup
import html


# extract_sustainable_info関数の定義
def extract_sustainable_info(fundcode, file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        xbrl_content = file.read()

    # エスケープ文字を元のタグに戻す
    decoded_content = html.unescape(xbrl_content)

    # BeautifulSoupでパース
    soup = BeautifulSoup(decoded_content, 'xml')

    # 各タグの属性情報を削除する
    def remove_tag_attributes(tag):
        for attribute in list(tag.attrs):
            del tag[attribute]

    # <tr>タグ、<td>タグで囲まれたテキストから改行コードを除去する
    def remove_newlines_within_tags(tag):
        if tag.name == 'tr':
            for td_tag in tag.find_all('td'):
                for child in td_tag.children:
                    if child.name == 'pstyle':  # <pstyle>タグは<p>に変換してスタイル属性を削除
                        child.name = 'p'
                        del child['style']
                # テキストノードの改行を除去
                td_tag.string = td_tag.get_text(separator="", strip=True)

    # すべてのタグを削除し、コンテンツのみを残す
    def remove_all_tags(tag):
        content_text = ''.join(str(child) for child in tag.contents if isinstance(child, BeautifulSoup))
        tag.extract()
        return content_text

    # 対象タグを定義
    element_tag = f'jpcrp030000-asr_{fundcode}-000:DisclosureOfSustainabilityRelatedFinancialInformationTextBlock'
    #print(element_tag)
    element_tag = f'jpcrp_cor:DisclosureOfSustainabilityRelatedFinancialInformationTextBlock'
    # 特定の要素を見つける
    elements = soup.find_all(element_tag)
    #print(elements)

    # テーブル内容の各要素に対して処理を適用
    processed_texts = []
    for i, elem in enumerate(elements):
        # 各タグの属性情報を削除する
        for tag in elem.find_all():
            remove_tag_attributes(tag)

        # <tr>タグ、<td>タグで囲まれたテキストから改行コードを除去する
        for tag in elem.find_all(['tr', 'td']):
            remove_newlines_within_tags(tag)

        # すべてのタグを削除し、コンテンツのみを残す
        processed_text = elem.get_text(separator="\n", strip=True)
        #print(processed_text)
        processed_texts.append(processed_text)
        #print(processed_texts)
#    return '\n'.join(processed_texts)
    return processed_texts

# 結果を格納するためのデータフレームを作成
extracted_df = pd.DataFrame(columns=['書類管理番号', 'EDINET（ファンド）コード', '企業名', 'XBRLファイル名', '抽出テキスト'])

# result_dfの各行に対してextract_sustainable_info関数を適用
rows_to_add = []
for index, row in result_df.iterrows():
    fundcode = row['EDINET(ファンド)コード']
    print(fundcode)
    file_path = row['XBRLファイルパス']
    print(file_path)
    extracted_text = extract_sustainable_info(fundcode, file_path)
    
    # 新しい行をリストに追加
    rows_to_add.append({
        '書類管理番号': row['書類管理番号'],
        'EDINET（ファンド）コード': row['EDINET(ファンド)コード'],
        '企業名': row['提出者(ファンド)'],
        'XBRLファイル名': row['XBRLファイル名'],
        '抽出テキスト': extracted_text
    })

# データフレームに変換して結合
extracted_df = pd.concat([extracted_df, pd.DataFrame(rows_to_add)], ignore_index=True)

# 結果のデータフレームを表示
print(extracted_df)
print(extracted_df.loc[10])


E00691
data/XBRL/Xbrl_Search_20240419_152847/S100R5K4/XBRL/PublicDoc/jpcrp030000-asr-001_E00691-000_2023-03-31_01_2023-06-28.xbrl
E00642
data/XBRL/Xbrl_Search_20240419_152909/S100R87F/XBRL/PublicDoc/jpcrp030000-asr-001_E00642-000_2023-03-31_01_2023-06-29.xbrl
E11873
data/XBRL/Xbrl_Search_20240419_152923/S100R8UQ/XBRL/PublicDoc/jpcrp030000-asr-001_E11873-000_2023-03-31_01_2023-06-29.xbrl
E00644
data/XBRL/Xbrl_Search_20240419_152939/S100R82J/XBRL/PublicDoc/jpcrp030000-asr-001_E00644-000_2023-03-31_01_2023-06-29.xbrl
E00645
data/XBRL/Xbrl_Search_20240419_152951/S100R6PJ/XBRL/PublicDoc/jpcrp030000-asr-001_E00645-000_2023-03-31_01_2023-06-29.xbrl
E00649
data/XBRL/Xbrl_Search_20240419_153004/S100R7Q8/XBRL/PublicDoc/jpcrp030000-asr-001_E00649-000_2023-03-31_01_2023-06-29.xbrl
E00660
data/XBRL/Xbrl_Search_20240419_153021/S100R8GL/XBRL/PublicDoc/jpcrp030000-asr-001_E00660-000_2023-03-31_01_2023-06-29.xbrl
E00659
data/XBRL/Xbrl_Search_20240419_153035/S100R69D/XBRL/PublicDoc/jpcrp030000-asr-001_E

In [100]:
print(fundcode)
print(file_path)
print(element_tag)
extract_sustainable_info(fundcode, file_path)

E01739
data/XBRL/Xbrl_Search_20240709_152741/S100TOGW/XBRL/PublicDoc/jpcrp030000-asr-001_E01739-000_2024-03-31_01_2024-06-25.xbrl
jpcrp030000-asr_E01739-000:DisclosureOfSustainabilityRelatedFinancialInformationTextBlock
